In [ ]:
import taichi as ti
from setup.voxel_setup import setup_voxel_scene
from simulation.simulator import *
from common.plot import Plotter
from data.octree import Octree
from data.mlp import *

# debug=True to check boundary access
ti.init(arch=ti.gpu)

import numpy as np
import matplotlib
from scipy import ndimage

%load_ext autoreload
%autoreload 2

# matplotlib.use('Qt5Agg')
%matplotlib widget

## 1. Load voxel model

In [ ]:
NUM_X, NUM_Y, NUM_Z = 128, 128, 128
scene, floor_height = setup_voxel_scene(NUM_X, NUM_Y, NUM_Z)

In [ ]:
# scene.finish()

## 2. Light simulation

In [ ]:
assert scene.ior.shape == (NUM_X, NUM_Y, NUM_Z), "The scene IOR should be a NumPy array of shape (NUM_X, NUM_Y, NUM_Z)"

sampler_multiplier = 3
pos_perturbation_scale = 0.45
initial_wavefront_pos, initial_wavefront_dir = generate_initial_wavefront(sampler_multiplier, pos_perturbation_scale, NUM_X, NUM_Y, NUM_Z)

scene.ior = ndimage.gaussian_filter(scene.ior, sigma=3.0, radius=1)
scene.gradient = compute_ior_gradient(scene.ior)

plotter = Plotter(sampler_multiplier, floor_height, scene.ior)
plotter.plot_wavefront_position(initial_wavefront_pos, initial_wavefront_dir, title="Initial Wavefront Positions")
plotter.plot_gradient(scene.gradient, threshold=0.01, alpha=0.01)

In [ ]:
test_delta_t = 0.3 * (NUM_Y / 100)
test_num_steps = int(1.1 * (NUM_Y / test_delta_t))
raw_irradiance, scene.local_diretion = simulate_wavefront_propagation(scene.ior, scene.gradient, scene.attenuation,
                                                initial_wavefront_pos, initial_wavefront_dir,
                                                plotter, test_num_steps, step_size=test_delta_t, num_show_images=0)

# plotter.plot_local_direction_grid_slices(scene.local_diretion, num_slices=8, z_start=30, z_end=120, stream_plot=False)
raw_irradiance = remove_under_floor(raw_irradiance, floor_height=floor_height)
scene.irradiance = ndimage.gaussian_filter(raw_irradiance, sigma=0.8) 

In [ ]:
plotter.plot_irradiance_slices(raw_irradiance, threshold=3, num_slices=8, z_start=30, z_end=120)
plotter.plot_irradiance_slices(scene.irradiance, threshold=3, num_slices=8, z_start=30, z_end=120)

## 3. Neural network irradiance （MLP）

In [ ]:
train_inputs, val_inputs, train_targets, val_targets = prepare_data(scene.irradiance, floor_height)
model = IrradianceNet().to(DEVICE)  
model = train_model(model, train_inputs, val_inputs, train_targets, val_targets, num_epochs=50)

In [ ]:
# Visualize the predicted irradiance field
predicted_irradiance = generate_irradiance_field_3d(model, floor_height, pad=True)
plotter.plot_irradiance_slices(predicted_irradiance.cpu().numpy(), threshold=3, num_slices=8, z_start=30, z_end=120)

## 4. Octree

In [ ]:
octree = Octree(threshold=int(0.6 * (sampler_multiplier**3)))
octree.construct(scene.irradiance)
print(f"Number of nodes: {len(octree)}")
print(f"Octree Memory usage: {octree.__sizeof__()} bytes")
print(f"In comparison, NumPy Storage Usage: {scene.irradiance.nbytes} bytes")
octree.visualize(plotter)

In [ ]:
x, y, z = 60, 60, 60
value = octree.query(x, y, z)
value

## 5. Ray marching

## Temp Tests

In [ ]:
@ti.data_oriented
class TiArray:
    def __init__(self, n):
        self.x = ti.field(dtype=ti.i32, shape=n)

    @ti.kernel
    def inc(self):
          
        for i in self.x:
            self.x[i] += 1

a = TiArray(32)
a.inc()
print(a.x.to_numpy())
print(a.x.dtype)